In [15]:
import concurrent.futures
from multiprocessing import shared_memory
import sys
from typing import Dict

import numpy as np

In [16]:
# create an array that can be used to represent a 2x2 cell lat/lon map with 3 times
prcp = np.array([[[1, 6, 5], [3, 2, 7]], [[8, 4., 6.], [9, 4, 2]]])
prcp.shape

(2, 2, 3)

In [17]:
# create a shared memory array for precipitation, copy the precipitation data
# into it, then replace the original underlying array with the shared memory
shm_prcp = shared_memory.SharedMemory(create=True, size=prcp.data.nbytes)
shared_prcp = np.ndarray(prcp.shape, dtype=prcp.dtype, buffer=shm_prcp.buf)
shared_prcp[:,:,:] = prcp[:,:,:]
shm_name_prcp = shm_prcp.name

In [18]:
def add_average(ary: np.ndarray) -> np.ndarray:
    return ary + np.mean(ary)

In [19]:
def shm_add_average(
    arguments: Dict,
):
    existing_shm = shared_memory.SharedMemory(name=arguments['shm_name'])
    shm_ary = np.ndarray(shape=arguments['shape'], dtype=arguments['dtype'], buffer=existing_shm.buf)
    shm_ary[arguments['lat_index'], arguments['lon_index']] = \
        add_average(shm_ary[arguments['lat_index'], arguments['lon_index']])

In [20]:
%%time
arguments_list = []
for lat_index in range(prcp.shape[0]):
    for lon_index in range(prcp.shape[0]):

        arguments = {
            'lat_index': lat_index,
            'lon_index': lon_index,
            'shm_name': shm_name_prcp,
            'dtype': shared_prcp.dtype,
            'shape': shared_prcp.shape
        }
        arguments_list.append(arguments)

# use a ProcessPoolExecutor to download the images in parallel
with concurrent.futures.ProcessPoolExecutor() as executor:

    # use the executor to map the download function to the iterable of arguments
    executor.map(shm_add_average, arguments_list)

CPU times: user 11.3 ms, sys: 13.6 ms, total: 24.9 ms
Wall time: 34.8 ms


In [21]:
shared_prcp

array([[[ 5., 10.,  9.],
        [ 7.,  6., 11.]],

       [[14., 10., 12.],
        [14.,  9.,  7.]]])

In [22]:
prcp

array([[[1., 6., 5.],
        [3., 2., 7.]],

       [[8., 4., 6.],
        [9., 4., 2.]]])

In [23]:
%%time
added_ary = prcp.copy()
arguments_list = []
for lat_index in range(prcp.shape[0]):
    for lon_index in range(prcp.shape[0]):
        added_ary[lat_index, lon_index] = add_average(prcp[lat_index, lon_index])
added_ary  

CPU times: user 233 µs, sys: 88 µs, total: 321 µs
Wall time: 199 µs


array([[[ 5., 10.,  9.],
        [ 7.,  6., 11.]],

       [[14., 10., 12.],
        [14.,  9.,  7.]]])

In [24]:
np.allclose(shared_prcp, added_ary)

True